In [89]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the tickers categorized by caps
categories = {
    "Large-Cap": [
        "ITOT", "IYY", "SCHB", "SPTM", "IVV", "SPY", "VOO", "SCHX", "DIA", "VTI", "VV"
    ],
    "Mid-Cap": [
        "MDY", "IJH", "VO", "SCHM", "SPMD"
    ],
    "Small-Cap": [
        "IJR", "VB", "SCHA", "VIOO", "SPSM"
    ],
    "Micro-Cap": [
        "BND", "SCHA", "SCHB", "SCHM", "SCHX", "SPMD", "SPSM", "SPTM", "VB", "VIOO", 
        "VTEB", "VV", "XLC", "XLRE"
    ]
}

# Define the date range
start_date = "2021-12-30"
end_date = "2024-11-30"

# Fetch historical price data for all tickers
all_tickers = [ticker for sublist in categories.values() for ticker in sublist]
fund_data = {}
print("Fetching historical data for all tickers...")
for ticker in all_tickers:
    try:
        print(f"Fetching data for {ticker}...")
        data = yf.download(ticker, start=start_date, end=end_date)[['Close']]
        if not data.empty:
            fund_data[ticker] = data
        else:
            print(f"No historical data available for {ticker}.")
    except Exception as e:
        print(f"Failed to fetch data for {ticker}: {e}")

# Combine all data into a single DataFrame with an additional 'Ticker' column
all_data = []
for ticker, data in fund_data.items():
    data['Ticker'] = ticker
    data['Date'] = data.index  # Ensure 'Date' column is present
    all_data.append(data)
human_fund = pd.concat(all_data)

# Ensure 'Date' is in datetime format
human_fund['Date'] = pd.to_datetime(human_fund['Date'])

# Drop funds with insufficient data
human_fund = human_fund.groupby('Ticker').filter(lambda x: len(x) >= 252)  # At least one year of data

# Function to compute equal variance weights
def compute_equal_variance_weights(returns):
    vols = returns.std()
    inv_vols = 1 / vols  # Inverse volatilities
    weights = inv_vols / inv_vols.sum()  # Normalize to sum to 1
    return weights

# Initialize a DataFrame to store daily portfolio values
portfolio_values_df = pd.DataFrame()

# Process each category
portfolio_metrics = []

for category, tickers_list in categories.items():
    print(f"Processing {category} funds...")
    category_data = human_fund[human_fund['Ticker'].isin(tickers_list)]

    if category_data.empty:
        print(f"No valid data for {category}. Skipping.")
        continue

    # Pivot close prices for return calculations
    close_prices = category_data.pivot(index='Date', columns='Ticker', values='Close').dropna()
    returns = close_prices.pct_change().dropna()

    if returns.empty or returns.shape[1] < 2:
        print(f"Not enough data for {category}.")
        continue

    # Initialize portfolio with a total capital of 1
    initial_capital = 1
    portfolio_value = initial_capital
    daily_values = [portfolio_value]
    portfolio_daily_returns = []

    # Iterate over each day with rebalancing
    for i in range(len(returns)):
        # Recompute weights based on equal variance contribution
        daily_returns = returns.iloc[:i+1]  # Include returns up to the current day
        stocks_vols = daily_returns.std()
        stocks_vols[stocks_vols == 0] = np.nan  # Replace zero volatilities with NaN
        stocks_vols = stocks_vols.dropna()      # Drop invalid volatilities
        if stocks_vols.empty:
            print(f"No valid volatilities on day {i}. Skipping.")
            daily_values.append(portfolio_value)  # Keep previous value
            continue

        inv_vols = 1 / stocks_vols  # Inverse volatilities
        weights = inv_vols / inv_vols.sum()  # Normalize weights to sum to 1

        # Calculate position values after rebalancing
        position_values = weights * portfolio_value

        # Update position values with daily returns
        position_values *= (1 + returns.iloc[i].fillna(0))  # Handle NaN safely

        # Calculate new portfolio value as the sum of position values
        portfolio_value = position_values.sum()
        daily_values.append(portfolio_value)

        # Calculate daily portfolio return
        if i > 0:  # Skip the first day, as there is no previous day to calculate returns
            portfolio_daily_returns.append((portfolio_value / daily_values[-2]) - 1)

    # Store daily portfolio values
    portfolio_values_df[category] = pd.Series(daily_values[1:], index=returns.index)  # Exclude the initial value


    # Calculate portfolio metrics
    portfolio_returns = np.array(portfolio_daily_returns)
    metrics = {
        "Annualized Return (%)": (1 + np.mean(portfolio_returns)) ** 252 - 1,
        "Sharpe Ratio": (np.mean(portfolio_returns) / np.std(portfolio_returns) * np.sqrt(252))
        if np.std(portfolio_returns) else np.nan,
        "Sortino Ratio": (np.mean(portfolio_returns) / np.std(portfolio_returns[portfolio_returns < 0]) * np.sqrt(252))
        if np.std(portfolio_returns[portfolio_returns < 0]) else np.nan,
        "Standard Deviation (%)": np.std(portfolio_returns) * np.sqrt(252),
        "Downside Deviation (%)": np.std(portfolio_returns[portfolio_returns < 0]) * np.sqrt(252)
        if np.std(portfolio_returns[portfolio_returns < 0]) else np.nan,
        "Category": category,
    }
    portfolio_metrics.append(metrics)

# Consolidate metrics into a single DataFrame
metrics_df = pd.DataFrame(portfolio_metrics)

# Save daily portfolio values and metrics to CSV
portfolio_values_df.to_csv("benchmark_values.csv")
metrics_df.to_csv("benchmark_metrics.csv", index=False)

Fetching historical data for all tickers...
Fetching data for ITOT...
[*********************100%%**********************]  1 of 1 completed
Fetching data for IYY...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SCHB...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SPTM...
[*********************100%%**********************]  1 of 1 completed
Fetching data for IVV...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SPY...
[*********************100%%**********************]  1 of 1 completed
Fetching data for VOO...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SCHX...
[*********************100%%**********************]  1 of 1 completed
Fetching data for DIA...
[*********************100%%**********************]  1 of 1 completed
Fetching data for VTI...
[*********************100%%**********************]  1 of 1 completed
Fetching dat

/var/folders/0p/fxg5ckl91l97lc30f5yd7h840000gn/T/ipykernel_38522/2358078256.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Ticker'] = ticker
/var/folders/0p/fxg5ckl91l97lc30f5yd7h840000gn/T/ipykernel_38522/2358078256.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'] = data.index  # Ensure 'Date' column is present


Processing Mid-Cap funds...
No valid volatilities on day 0. Skipping.
Processing Small-Cap funds...
No valid volatilities on day 0. Skipping.
Processing Micro-Cap funds...
No valid volatilities on day 0. Skipping.


In [90]:
portfolio_values_df

,Large-Cap,Mid-Cap,Small-Cap,Micro-Cap
Date,,,,
2021-12-31,1.000000,1.000000,1.000000,1.000000
2022-01-03,1.005136,1.002080,1.009535,1.000573
2022-01-04,1.004691,1.005794,1.014312,1.001286
2022-01-05,0.985434,0.981962,0.986968,0.993116
2022-01-06,0.984176,0.986145,0.992827,0.993073
...,...,...,...,...
2024-11-22,1.240842,1.152377,1.102523,1.070819
2024-11-25,1.246433,1.167941,1.122250,1.080286
2024-11-26,1.252202,1.163689,1.112753,1.079762


In [91]:
metrics_df

,Annualized Return (%),Sharpe Ratio,Sortino Ratio,Standard Deviation (%),Downside Deviation (%),Category
0,0.097844,0.537716,0.768894,0.173634,0.121428,Large-Cap
1,0.074368,0.361135,0.556880,0.198659,0.128830,Mid-Cap
2,0.063363,0.280864,0.463369,0.218769,0.132603,Small-Cap
3,0.034285,0.302142,0.474822,0.111578,0.071000,Micro-Cap
